## 📖 Teoria: Operazioni Bitwise

### Gli Operatori

```c
<<   Left shift    (sposta bit a sinistra)
>>   Right shift   (sposta bit a destra)
&    AND bitwise   (1 solo se entrambi 1)
|    OR bitwise    (1 se almeno uno è 1)
~    NOT bitwise   (inverte tutti i bit)
^    XOR bitwise   (1 se diversi)
```

### Shift Left (<<)

```
uint8_t x = 3;      // 00000011
x << 1;             // 00000110 (= 6)
x << 2;             // 00001100 (= 12)

Regola: x << n  =  x * 2^n
```

### Shift Right (>>)

```
uint8_t x = 12;     // 00001100
x >> 1;             // 00000110 (= 6)
x >> 2;             // 00000011 (= 3)

Regola: x >> n  =  x / 2^n
```

### AND bitwise (&)

```
  10110011
& 11110000
  --------
  10110000

Uso: ESTRARRE bit specifici (masking)
```

### OR bitwise (|)

```
  10110011
| 00001100
  --------
  10111111

Uso: COMBINARE valori
```

## 🔢 Rappresentazione IP: 192.168.1.1

### Formato Dotted Decimal

```
"192.168.1.1"
  ^   ^   ^ ^
  |   |   | └─ Ottetto 4 (0-255)
  |   |   └─── Ottetto 3
  |   └─────── Ottetto 2
  └─────────── Ottetto 1
```

### Formato uint32_t (Network Byte Order = Big Endian)

```
uint32_t ip = 0xC0A80101

Bit layout (32 bit):
┌────────┬────────┬────────┬────────┐
│11000000│10101000│00000001│00000001│
└────────┴────────┴────────┴────────┘
   192      168       1        1
  bit      bit      bit      bit
  31-24    23-16    15-8     7-0
```

### Conversione Decimale

```
192 = 0xC0 = 11000000
168 = 0xA8 = 10101000
  1 = 0x01 = 00000001
  1 = 0x01 = 00000001

Decimale: 192*256³ + 168*256² + 1*256 + 1 = 3232235777
```

## 🛠️ Funzione 1: string_to_ip()

### Algoritmo

```
Input: "192.168.1.1"

1. Parse 4 numeri con sscanf()
2. Valida range (0-255)
3. Componi uint32_t:
   - 192 nei bit 31-24 (shift left 24)
   - 168 nei bit 23-16 (shift left 16)
   - 1 nei bit 15-8 (shift left 8)
   - 1 nei bit 7-0 (no shift)
4. Combina con OR

Output: 0xC0A80101
```

### Visualizzazione Step-by-Step

```
Step 1: 192 << 24
┌────────┬────────┬────────┬────────┐
│11000000│00000000│00000000│00000000│ = 0xC0000000
└────────┴────────┴────────┴────────┘

Step 2: 168 << 16
┌────────┬────────┬────────┬────────┐
│00000000│10101000│00000000│00000000│ = 0x00A80000
└────────┴────────┴────────┴────────┘

Step 3: 1 << 8
┌────────┬────────┬────────┬────────┐
│00000000│00000000│00000001│00000000│ = 0x00000100
└────────┴────────┴────────┴────────┘

Step 4: 1 (no shift)
┌────────┬────────┬────────┬────────┐
│00000000│00000000│00000000│00000001│ = 0x00000001
└────────┴────────┴────────┴────────┘

Step 5: OR tutti insieme
  0xC0000000
| 0x00A80000
| 0x00000100
| 0x00000001
  ----------
  0xC0A80101 ✓
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/02
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/02

# Copia ip_types.h dal notebook precedente
cp ../01/ip_types.h .

cat > ip_utils.h << 'EOF'
#ifndef IP_UTILS_H
#define IP_UTILS_H

#include "ip_types.h"
#include <stdbool.h>
#include <stdint.h>

// Conversioni base
bool string_to_ip(const char *str, uint32_t *ip);
void ip_to_string(uint32_t ip, char *buffer);
void ip_to_octets(uint32_t ip, uint8_t octets[4]);
uint32_t octets_to_ip(const uint8_t octets[4]);

#endif
EOF

cat > ip_utils.c << 'EOF'
#include "ip_utils.h"
#include <stdio.h>
#include <string.h>

/**
 * Converte stringa "192.168.1.1" in uint32_t
 * 
 * Algoritmo:
 * 1. Usa sscanf per parsare 4 numeri
 * 2. Valida range (0-255)
 * 3. Shifta ogni ottetto nella posizione corretta
 * 4. Combina con OR
 */
bool string_to_ip(const char *str, uint32_t *ip) {
    if (!str || !ip) return false;
    
    unsigned int oct1, oct2, oct3, oct4;
    
    // sscanf ritorna il numero di elementi letti
    if (sscanf(str, "%u.%u.%u.%u", &oct1, &oct2, &oct3, &oct4) != 4) {
        return false;  // Formato non valido
    }
    
    // Validazione: ogni ottetto deve essere 0-255
    if (oct1 > 255 || oct2 > 255 || oct3 > 255 || oct4 > 255) {
        return false;
    }
    
    // Combinazione: shift + OR
    // oct1 nei bit 31-24, oct2 nei 23-16, oct3 nei 15-8, oct4 nei 7-0
    *ip = ((uint32_t)oct1 << 24) |  // 0xXX000000
          ((uint32_t)oct2 << 16) |  // 0x00XX0000
          ((uint32_t)oct3 << 8)  |  // 0x0000XX00
          (uint32_t)oct4;           // 0x000000XX
    
    return true;
}

/**
 * Converte uint32_t in stringa "192.168.1.1"
 * 
 * Algoritmo:
 * 1. Estrai ogni ottetto con shift + mask
 * 2. Formatta con sprintf
 */
void ip_to_string(uint32_t ip, char *buffer) {
    if (!buffer) return;
    
    // Estrazione ottetti:
    // - Shift right per portare l'ottetto nei bit 7-0
    // - Mask & 0xFF per estrarre solo gli 8 bit meno significativi
    
    uint8_t oct1 = (ip >> 24) & 0xFF;  // Bit 31-24
    uint8_t oct2 = (ip >> 16) & 0xFF;  // Bit 23-16
    uint8_t oct3 = (ip >> 8) & 0xFF;   // Bit 15-8
    uint8_t oct4 = ip & 0xFF;          // Bit 7-0 (no shift needed)
    
    // Formattazione
    sprintf(buffer, "%u.%u.%u.%u", oct1, oct2, oct3, oct4);
}

/**
 * Estrae i 4 ottetti da un IP
 */
void ip_to_octets(uint32_t ip, uint8_t octets[4]) {
    if (!octets) return;
    
    octets[0] = (ip >> 24) & 0xFF;
    octets[1] = (ip >> 16) & 0xFF;
    octets[2] = (ip >> 8) & 0xFF;
    octets[3] = ip & 0xFF;
}

/**
 * Compone un IP da 4 ottetti
 */
uint32_t octets_to_ip(const uint8_t octets[4]) {
    if (!octets) return 0;
    
    return ((uint32_t)octets[0] << 24) |
           ((uint32_t)octets[1] << 16) |
           ((uint32_t)octets[2] << 8) |
           (uint32_t)octets[3];
}
EOF

echo "✅ File creati: ip_utils.h e ip_utils.c"

## ✅ Test delle Conversioni

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/02

cat > test_conversions.c << 'EOF'
#include <stdio.h>
#include "ip_utils.h"

void test_string_to_ip() {
    printf("=== TEST string_to_ip() ===\n");
    
    const char *test_cases[] = {
        "192.168.1.1",
        "10.0.0.1",
        "255.255.255.255",
        "8.8.8.8",
        "127.0.0.1"
    };
    
    for (int i = 0; i < 5; i++) {
        uint32_t ip;
        if (string_to_ip(test_cases[i], &ip)) {
            printf("  %-15s → 0x%08X (%u)\n", 
                   test_cases[i], ip, ip);
        } else {
            printf("  %-15s → ERRORE\n", test_cases[i]);
        }
    }
    printf("\n");
}

void test_ip_to_string() {
    printf("=== TEST ip_to_string() ===\n");
    
    uint32_t test_ips[] = {
        0xC0A80101,  // 192.168.1.1
        0x0A000001,  // 10.0.0.1
        0xFFFFFFFF,  // 255.255.255.255
        0x08080808,  // 8.8.8.8
        0x7F000001   // 127.0.0.1
    };
    
    for (int i = 0; i < 5; i++) {
        char buffer[16];
        ip_to_string(test_ips[i], buffer);
        printf("  0x%08X → %s\n", test_ips[i], buffer);
    }
    printf("\n");
}

void test_roundtrip() {
    printf("=== TEST ROUNDTRIP (string→ip→string) ===\n");
    
    const char *original = "192.168.1.1";
    uint32_t ip;
    char result[16];
    
    printf("  Original: %s\n", original);
    
    if (string_to_ip(original, &ip)) {
        printf("  IP num:   0x%08X\n", ip);
        ip_to_string(ip, result);
        printf("  Result:   %s\n", result);
        printf("  Match:    %s\n\n", 
               strcmp(original, result) == 0 ? "✓" : "✗");
    }
}

void test_octets() {
    printf("=== TEST Octets ===\n");
    
    uint32_t ip = 0xC0A80101;
    uint8_t octets[4];
    
    printf("  IP: 0x%08X\n", ip);
    
    ip_to_octets(ip, octets);
    printf("  Octets: [%u, %u, %u, %u]\n", 
           octets[0], octets[1], octets[2], octets[3]);
    
    uint32_t reconstructed = octets_to_ip(octets);
    printf("  Ricostruito: 0x%08X\n", reconstructed);
    printf("  Match: %s\n\n", ip == reconstructed ? "✓" : "✗");
}

void test_bitwise_visualization() {
    printf("=== VISUALIZZAZIONE BITWISE ===\n");
    printf("IP: 192.168.1.1 = 0xC0A80101\n\n");
    
    uint32_t ip = 0xC0A80101;
    
    printf("Ottetto 1 (192 = 0xC0):\n");
    printf("  ip >> 24 = 0x%08X\n", ip >> 24);
    printf("  & 0xFF   = 0x%02X = %u\n\n", (ip >> 24) & 0xFF, (ip >> 24) & 0xFF);
    
    printf("Ottetto 2 (168 = 0xA8):\n");
    printf("  ip >> 16 = 0x%08X\n", ip >> 16);
    printf("  & 0xFF   = 0x%02X = %u\n\n", (ip >> 16) & 0xFF, (ip >> 16) & 0xFF);
    
    printf("Composizione:\n");
    printf("  192 << 24 = 0x%08X\n", 192u << 24);
    printf("  168 << 16 = 0x%08X\n", 168u << 16);
    printf("    1 <<  8 = 0x%08X\n", 1u << 8);
    printf("    1 <<  0 = 0x%08X\n", 1u);
    printf("  ----------------------\n");
    printf("  OR tutti  = 0x%08X\n", 
           (192u << 24) | (168u << 16) | (1u << 8) | 1u);
}

int main() {
    test_string_to_ip();
    test_ip_to_string();
    test_roundtrip();
    test_octets();
    test_bitwise_visualization();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_conversions ip_utils.c test_conversions.c && ./test_conversions

## 🔍 Analisi Dettagliata: Perché & 0xFF?

### Il Problema

```c
uint32_t ip = 0xC0A80101;
uint8_t oct1 = ip >> 24;  // Cosa succede?
```

Dopo `ip >> 24`:

```
Prima:  11000000 10101000 00000001 00000001
Dopo:   00000000 00000000 00000000 11000000
        └────────────────────────┴────────┘
               24 zeri             192
```

Il valore è `0x000000C0` (uint32_t), ma vogliamo solo `0xC0` (uint8_t).

### La Soluzione: Mask & 0xFF

```
  00000000 00000000 00000000 11000000  (ip >> 24)
& 00000000 00000000 00000000 11111111  (0xFF)
  ------------------------------------
  00000000 00000000 00000000 11000000  (result)
```

`0xFF` = maschera che mantiene solo gli 8 bit meno significativi.

### Perché è Necessario?

1. **Type safety**: converte da uint32_t a uint8_t in modo esplicito
2. **Portabilità**: comportamento predicibile su tutte le architetture
3. **Chiarezza**: documenta l'intenzione di estrarre 1 byte

## ⚠️ Errori Comuni e Come Evitarli

### Errore 1: Overflow nell'OR

```c
// MALE: oct1 potrebbe essere promoted a int (signed)
uint32_t ip = (oct1 << 24) | (oct2 << 16) | ...;

// BENE: cast esplicito a uint32_t
uint32_t ip = ((uint32_t)oct1 << 24) | ...;
```

**Perché?** Se `oct1` è `unsigned int` e shift supera 31 bit, comportamento undefined.

### Errore 2: Dimenticare la Validazione

```c
// MALE: nessun controllo
sscanf(str, "%u.%u.%u.%u", &o1, &o2, &o3, &o4);
// E se str = "300.1.1.1"? → overflow silenzioso!

// BENE: valida range
if (o1 > 255 || o2 > 255 || ...) return false;
```

### Errore 3: Buffer Overflow

```c
// MALE: buffer troppo piccolo
char buf[15];  // "255.255.255.255" = 15 char + \0 = 16!
ip_to_string(ip, buf);  // OVERFLOW!

// BENE: buffer giusto
char buf[16];  // OK
```

### Errore 4: Pointer NULL

```c
// BENE: sempre controllare
if (!str || !ip) return false;
```

## 🎓 Concetti Avanzati

### Network Byte Order vs Host Byte Order

```c
#include <arpa/inet.h>

uint32_t host = 0x0A000001;      // 10.0.0.1 in host order
uint32_t net = htonl(host);      // Converti a network order
uint32_t back = ntohl(net);      // Torna a host order

// htonl = Host TO Network Long
// ntohl = Network TO Host Long
```

**Big endian (network)**: byte più significativo prima  
**Little endian (x86)**: byte meno significativo prima

Nel nostro caso, usiamo **sempre big endian** (network byte order) per semplicità.

### Alternativa con Union

```c
typedef union {
    uint32_t num;
    uint8_t bytes[4];
} IP;

IP ip;
ip.num = 0xC0A80101;
// Accesso diretto: ip.bytes[0], ip.bytes[1], ...
```

**Pro**: nessuna conversione necessaria  
**Contro**: dipendente da endianness della macchina

## 📚 Recap

### ✅ Funzioni implementate:

1. `string_to_ip()` - Parsing "192.168.1.1" → 0xC0A80101
2. `ip_to_string()` - Formatting 0xC0A80101 → "192.168.1.1"
3. `ip_to_octets()` - Estrazione [192, 168, 1, 1]
4. `octets_to_ip()` - Composizione da array

### 🎓 Operazioni bitwise apprese:

- **Shift left** (`<<`): posizionare byte
- **Shift right** (`>>`): estrarre byte
- **AND** (`&`): mascherare bit
- **OR** (`|`): combinare valori

### 🔑 Concetti chiave:

```
Estrazione:  (ip >> shift) & 0xFF
Composizione: (byte << shift) | ...
Validazione: sempre controllare range e NULL
```

### ➡️ Prossimo Notebook

**Notebook 3: Calcoli di Subnet**
- `create_netmask()` - Da /24 a 255.255.255.0
- `calculate_network_address()` - Primo IP subnet
- `calculate_broadcast_address()` - Ultimo IP subnet
- `ip_in_subnet()` - Verifica appartenenza
- `count_available_hosts()` - Quanti IP disponibili

---

## 🎉 Notebook 2 Completato!

**File creati**: `ip_utils.h`, `ip_utils.c`  
**Funzioni**: 4 conversioni fondamentali  
**Concetti**: Bitwise operators, masking, parsing

Continua con il Notebook 3! 🚀